In [23]:
import pandas as pd
from pymongo import MongoClient
from dotenv import load_dotenv
import os
from tqdm import tqdm
from langchain.chat_models import ChatOpenAI
from langchain.chains.llm import LLMChain
from langchain.prompts import PromptTemplate
from langchain.chains.combine_documents.stuff import StuffDocumentsChain
import requests
import glob
from bs4 import BeautifulSoup

load_dotenv()

True

In [5]:
MONGO_CONN=os.environ.get("MONGO_CONNECTION_STRING")
client = MongoClient(MONGO_CONN)
col = client["bfsi-genai"]["credit_history"]
vcol = client["bfsi-genai"]["cc_products"]

# Define prompt
prompt_template = """Write a concise summary of the following in exactly 4000 words:
"{text}"
Point by Point SUMMARY of Credit card mentioned Above:"""
prompt = PromptTemplate.from_template(prompt_template)
llm = ChatOpenAI(temperature=0, model_name="gpt-3.5-turbo-16k")
llm_chain = LLMChain(llm=llm, prompt=prompt)


# Define StuffDocumentsChain
stuff_chain = StuffDocumentsChain(
    llm_chain=llm_chain, document_variable_name="text"
)

In [12]:
text="""
Our Credit Cards are like no other:
Let's find a Card best suited for you!


Lifetime free

Low interest rates
starting from 9% p.a.

Upto 10X rewards
10X rewards on your birthday, and crossing the spend threshold, on select variants

Interest free
ATM cash withdrawal for upto 48 days
+ more reasons to love the card


"""
temp =f"""Write a concise summary of the following in exactly 4000 words:
"{text}"
Point by Point SUMMARY of Credit card mentioned Above:"""


# print(temp)
# print(llm.predict(temp))

In [28]:
# df = pd.read_csv("../data/cs-training.csv")
# df["MonthlyIncome"] = df["MonthlyIncome"]*25

# recs = df.to_dict(orient="records")
# col.insert_many(recs)


In [25]:
from langchain.docstore.document import Document
url = "https://www.tengenbank.com/personal/pay/cards/credit-cards/marriott-bonvoy-credit-card"
def parser(url,i=-1):
    response = requests.get(url)
    page_content = response.content
    soup = BeautifulSoup(page_content, 'html.parser')
    products = soup.find('div', {"class": "main-content"})
    return [Document(metadata={"source": url,"title": url.split("/")[i]},page_content=products.text)]

parser(url)

[Document(page_content='\nFeatures\n\n\n\n\n\n\nReward Type\n\nReward Type\n\n\n\n\nMarriott Bonvoy Points\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nFeatures\n\nFeatures\n\n\n\n\n1 Free Night Award on Joining fee realization10 Elite Night Credits (ENCs) under the Marriott Bonvoy ProgramComplimentary Marriott Bonvoy Silver Elite StatusEarn 8 Marriott Bonvoy Points per Rs. 150 spent at hotels participating in Marriott Bonvoy*Earn 4 Marriott Bonvoy Points per Rs. 150 spent on travel, dining & entertainment*Earn 2 Marriott Bonvoy Points per Rs. 150 spent on all other applicable purchases*Complimentary Golf Access 2 per quarter across the world (Green Fee Waiver)Marriott Bonvoy Points will not be accrued for the following non-core product features categories:FuelSmart EMI / Dial an EMI transactionWallet loads / Gift or Prepaid Card load/ Voucher purchaseCash AdvancesPayment of Outstanding BalancesPayment of card fees and other chargesGovernment related transactions & Rental transactions*Click 

In [15]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 1000,
    chunk_overlap  = 100,
)

def parse_n_load(url):
    op = []
    try:
        op = parser(url)
        try:
            op += parser(url+"/eligibility",-2)
        except:
            print(f"Failed to Parse Eligibility {url}")
        op += parser(url+"/fees-and-charges",-2)
    except:
        print(f"Failed to Parse Entire {url}")
    return op

def summarize_docs(docs):
    return stuff_chain.run(docs)

def get_data(url):
    data = parse_n_load(url)
    docs = text_splitter.split_documents(data)
    summary = summarize_docs(docs)
    doc = [Document(metadata={"source": url,"title": url.split("/")[-1]},page_content=summary)]
    return doc

# Split the long text and Summarize

In [63]:
results = ["https://www.hdfcbank.com/personal/pay/cards/credit-cards/marriott-bonvoy-credit-card","https://www.hdfcbank.com/personal/pay/cards/credit-cards/swiggy-hdfc-bank-credit-card","https://www.hdfcbank.com/personal/pay/cards/credit-cards/regalia-gold-credit-card","https://www.hdfcbank.com/personal/pay/cards/credit-cards/irctc-credit-card","https://www.hdfcbank.com/personal/pay/cards/credit-cards/hdfc-bank-upi-rupay-credit-card","https://www.hdfcbank.com/personal/pay/cards/credit-cards/diners-privilege","https://www.hdfcbank.com/personal/pay/cards/credit-cards/moneyback-plus","https://www.hdfcbank.com/personal/pay/cards/credit-cards/freedom-card-new","https://www.hdfcbank.com/personal/pay/cards/credit-cards/indianoil-hdfc-bank-credit-card","https://www.hdfcbank.com/personal/pay/cards/credit-cards/infinia-credit-card","https://www.hdfcbank.com/personal/pay/cards/credit-cards/diners-club-black","https://www.hdfcbank.com/personal/pay/cards/credit-cards/tata-neu-infinity-hdfc-bank-credit-card","https://www.hdfcbank.com/personal/pay/cards/credit-cards/tata-neu-plus-hdfc-bank-credit-card","https://www.hdfcbank.com/personal/pay/cards/credit-cards/shoppers-stop-hdfc-bank-credit-card","https://www.hdfcbank.com/personal/pay/cards/credit-cards/shoppers-stop-black-hdfc-bank-credit-card","https://www.hdfcbank.com/personal/pay/cards/credit-cards/paytm-hdfc-bank-credit-card","https://www.hdfcbank.com/personal/pay/cards/credit-cards/paytm-hdfc-bank-select-credit-card","https://www.hdfcbank.com/personal/pay/cards/credit-cards/paytm-hdfc-bank-mobile-credit-card","https://www.hdfcbank.com/personal/pay/cards/credit-cards/paytm-hdfc-bank-digital-credit-card","https://www.hdfcbank.com/personal/pay/cards/credit-cards/platinum-times-credit-card","https://www.hdfcbank.com/personal/pay/cards/credit-cards/titanium-times-credit-card","https://www.hdfcbank.com/personal/pay/cards/credit-cards/6e-rewards-indigo-hdfc-bank-credit-card","https://www.hdfcbank.com/personal/pay/cards/credit-cards/6e-rewards-xl-indigo-hdfc-bank-credit-card","https://www.hdfcbank.com/personal/pay/cards/credit-cards/all-miles-card","https://www.hdfcbank.com/personal/pay/cards/credit-cards/freedom-card","https://www.hdfcbank.com/personal/pay/cards/credit-cards/bharat-credit-card","https://www.hdfcbank.com/personal/pay/cards/credit-cards/diners-club-premium","https://www.hdfcbank.com/personal/pay/cards/credit-cards/diners-club-rewardz","https://www.hdfcbank.com/personal/pay/cards/credit-cards/doctors-regalia","https://www.hdfcbank.com/personal/pay/cards/credit-cards/doctors-superia","https://www.hdfcbank.com/personal/pay/cards/credit-cards/jetprivilege-hdfc-bank-titanium-credit-card","https://www.hdfcbank.com/personal/pay/cards/credit-cards/platinum-edge","https://www.hdfcbank.com/personal/pay/cards/credit-cards/platinum-plus","https://www.hdfcbank.com/personal/pay/cards/credit-cards/solitaire","https://www.hdfcbank.com/personal/pay/cards/credit-cards/superia","https://www.hdfcbank.com/personal/pay/cards/credit-cards/teachers-platinum","https://www.hdfcbank.com/personal/pay/cards/credit-cards/titanium-edge","https://www.hdfcbank.com/personal/pay/cards/credit-cards/visa-signature","https://www.hdfcbank.com/personal/pay/cards/credit-cards/world-mastercard","https://www.hdfcbank.com/personal/pay/cards/credit-cards/money-back","https://www.hdfcbank.com/personal/pay/cards/credit-cards/regalia-first","https://www.hdfcbank.com/personal/pay/cards/credit-cards/diners-club-miles","https://www.hdfcbank.com/personal/pay/cards/credit-cards/regalia","https://www.hdfcbank.com/personal/pay/cards/credit-cards/jet-privilege-hdfc-bank-diners-club","https://www.hdfcbank.com/personal/pay/cards/credit-cards/jet-privilege-hdfc-bank-world","https://www.hdfcbank.com/personal/pay/cards/credit-cards/jetprivilege-hdfc-bank-platinum-credit-card","https://www.hdfcbank.com/personal/pay/cards/credit-cards/diners-privilege-old"]
# url = "https://www.hdfcbank.com/personal/pay/cards/credit-cards/marriott-bonvoy-credit-card"
def to_file(url):
    op = parser(url)
    op += parser(url+"/eligibility",-2)
    op += parser(url+"/fees-and-charges",-2)
    # details = text_splitter.split_documents(op)
    final = "\n".join(" ".join(detail.page_content.split()) for detail in op)
    path = f"/Users/ashwin.gangadhar/projects/mdb-bfsi-genai/data/cards-data/{op[0].metadata['title']}.txt"
    print(path)
    with open(path,'w') as fw:
        fw.write(final)
to_file(results[0])

/Users/ashwin.gangadhar/projects/mdb-bfsi-genai/data/cards-data/marriott-bonvoy-credit-card.txt


In [42]:
print(summarize_docs(details).replace(".", ".\n"))


The Marriott Bonvoy 10gen Bank Credit Card offers various features and rewards.
 Upon joining, cardholders receive a free night award, 10 Elite Night Credits, and complimentary Marriott Bonvoy Silver Elite Status.
 They can earn 8 Marriott Bonvoy Points per Rs.
 150 spent at participating hotels, 4 points per Rs.
 150 spent on travel, dining, and entertainment, and 2 points per Rs.
 150 spent on other purchases.
 Cardholders also receive complimentary golf access and lounge access within India and internationally.
 Additional features include zero lost card liability and a foreign currency markup of 3.
5%.
 Milestone benefits include free night awards based on eligible spend.
 The card also offers insurance coverage and the ability to redeem points for stays, experiences, and travel.
 Cardholders can access concierge services, use Smart EMI for converting big spends into EMI, and make contactless payments.
 The eligibility criteria for the card include a minimum age of 21 years and a g

In [2]:
docs = []
results = ["https://www.tengenbank.com/personal/pay/cards/credit-cards/marriott-bonvoy-credit-card","https://www.tengenbank.com/personal/pay/cards/credit-cards/swiggy-hdfc-bank-credit-card","https://www.tengenbank.com/personal/pay/cards/credit-cards/regalia-gold-credit-card","https://www.tengenbank.com/personal/pay/cards/credit-cards/irctc-credit-card","https://www.tengenbank.com/personal/pay/cards/credit-cards/hdfc-bank-upi-rupay-credit-card","https://www.tengenbank.com/personal/pay/cards/credit-cards/diners-privilege","https://www.tengenbank.com/personal/pay/cards/credit-cards/moneyback-plus","https://www.tengenbank.com/personal/pay/cards/credit-cards/freedom-card-new","https://www.tengenbank.com/personal/pay/cards/credit-cards/indianoil-hdfc-bank-credit-card","https://www.tengenbank.com/personal/pay/cards/credit-cards/infinia-credit-card","https://www.tengenbank.com/personal/pay/cards/credit-cards/diners-club-black","https://www.tengenbank.com/personal/pay/cards/credit-cards/tata-neu-infinity-hdfc-bank-credit-card","https://www.tengenbank.com/personal/pay/cards/credit-cards/tata-neu-plus-hdfc-bank-credit-card","https://www.tengenbank.com/personal/pay/cards/credit-cards/shoppers-stop-hdfc-bank-credit-card","https://www.tengenbank.com/personal/pay/cards/credit-cards/shoppers-stop-black-hdfc-bank-credit-card","https://www.tengenbank.com/personal/pay/cards/credit-cards/paytm-hdfc-bank-credit-card","https://www.tengenbank.com/personal/pay/cards/credit-cards/paytm-hdfc-bank-select-credit-card","https://www.tengenbank.com/personal/pay/cards/credit-cards/paytm-hdfc-bank-mobile-credit-card","https://www.tengenbank.com/personal/pay/cards/credit-cards/paytm-hdfc-bank-digital-credit-card","https://www.tengenbank.com/personal/pay/cards/credit-cards/platinum-times-credit-card","https://www.tengenbank.com/personal/pay/cards/credit-cards/titanium-times-credit-card","https://www.tengenbank.com/personal/pay/cards/credit-cards/6e-rewards-indigo-hdfc-bank-credit-card","https://www.tengenbank.com/personal/pay/cards/credit-cards/6e-rewards-xl-indigo-hdfc-bank-credit-card","https://www.tengenbank.com/personal/pay/cards/credit-cards/all-miles-card","https://www.tengenbank.com/personal/pay/cards/credit-cards/freedom-card","https://www.tengenbank.com/personal/pay/cards/credit-cards/bharat-credit-card","https://www.tengenbank.com/personal/pay/cards/credit-cards/diners-club-premium","https://www.tengenbank.com/personal/pay/cards/credit-cards/diners-club-rewardz","https://www.tengenbank.com/personal/pay/cards/credit-cards/doctors-regalia","https://www.tengenbank.com/personal/pay/cards/credit-cards/doctors-superia","https://www.tengenbank.com/personal/pay/cards/credit-cards/jetprivilege-hdfc-bank-titanium-credit-card","https://www.tengenbank.com/personal/pay/cards/credit-cards/platinum-edge","https://www.tengenbank.com/personal/pay/cards/credit-cards/platinum-plus","https://www.tengenbank.com/personal/pay/cards/credit-cards/solitaire","https://www.tengenbank.com/personal/pay/cards/credit-cards/superia","https://www.tengenbank.com/personal/pay/cards/credit-cards/teachers-platinum","https://www.tengenbank.com/personal/pay/cards/credit-cards/titanium-edge","https://www.tengenbank.com/personal/pay/cards/credit-cards/visa-signature","https://www.tengenbank.com/personal/pay/cards/credit-cards/world-mastercard","https://www.tengenbank.com/personal/pay/cards/credit-cards/money-back","https://www.tengenbank.com/personal/pay/cards/credit-cards/regalia-first","https://www.tengenbank.com/personal/pay/cards/credit-cards/diners-club-miles","https://www.tengenbank.com/personal/pay/cards/credit-cards/regalia","https://www.tengenbank.com/personal/pay/cards/credit-cards/jet-privilege-hdfc-bank-diners-club","https://www.tengenbank.com/personal/pay/cards/credit-cards/jet-privilege-hdfc-bank-world","https://www.tengenbank.com/personal/pay/cards/credit-cards/jetprivilege-hdfc-bank-platinum-credit-card","https://www.tengenbank.com/personal/pay/cards/credit-cards/diners-privilege-old"]
for result in tqdm(results):
    docs += get_data(result)

# Generate embedding to the summarized documents

In [16]:
from langchain.embeddings.huggingface import HuggingFaceInstructEmbeddings
from langchain.vectorstores import MongoDBAtlasVectorSearch
repo_id = "hkunlp/instructor-base"
hf = HuggingFaceInstructEmbeddings(model_name=repo_id, cache_folder="tmp/")
hf.embed_instruction = "Represent the document for retrieval of credit cards:"
vectorstore = MongoDBAtlasVectorSearch(vcol, hf)

/Users/ashwin.gangadhar/anaconda3/lib/python3.9/site-packages/InstructorEmbedding/instructor.py:7: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import trange


load INSTRUCTOR_Transformer
max_seq_length  512


In [45]:
# vcol.delete_many({})
# vectorstore.add_documents(docs)

[ObjectId('6517ebd2aba949c4b64d3bbe'),
 ObjectId('6517ebd2aba949c4b64d3bbf'),
 ObjectId('6517ebd2aba949c4b64d3bc0'),
 ObjectId('6517ebd2aba949c4b64d3bc1'),
 ObjectId('6517ebd2aba949c4b64d3bc2'),
 ObjectId('6517ebd2aba949c4b64d3bc3'),
 ObjectId('6517ebd2aba949c4b64d3bc4'),
 ObjectId('6517ebd2aba949c4b64d3bc5'),
 ObjectId('6517ebd2aba949c4b64d3bc6'),
 ObjectId('6517ebd2aba949c4b64d3bc7'),
 ObjectId('6517ebd2aba949c4b64d3bc8'),
 ObjectId('6517ebd2aba949c4b64d3bc9'),
 ObjectId('6517ebd2aba949c4b64d3bca'),
 ObjectId('6517ebd2aba949c4b64d3bcb'),
 ObjectId('6517ebd2aba949c4b64d3bcc'),
 ObjectId('6517ebd2aba949c4b64d3bcd'),
 ObjectId('6517ebd2aba949c4b64d3bce'),
 ObjectId('6517ebd2aba949c4b64d3bcf'),
 ObjectId('6517ebd2aba949c4b64d3bd0'),
 ObjectId('6517ebd2aba949c4b64d3bd1'),
 ObjectId('6517ebd2aba949c4b64d3bd2'),
 ObjectId('6517ebd2aba949c4b64d3bd3'),
 ObjectId('6517ebd2aba949c4b64d3bd4'),
 ObjectId('6517ebd2aba949c4b64d3bd5'),
 ObjectId('6517ebd2aba949c4b64d3bd6'),
 ObjectId('6517ebd2aba949

In [22]:
docs = vectorstore.as_retriever().get_relevant_documents("Credit card that provide great value for travelling and vacation")
for doc in docs:
    print(doc.metadata["title"])

diners-club-miles
visa-signature
world-mastercard
diners-club-premium


In [29]:
from sentence_transformers import SentenceTransformer
sentences = ["This is an example sentence", "Each sentence is converted"]

model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')
embeddings = model.encode(sentences)
print(embeddings)

[[ 6.76569194e-02  6.34958595e-02  4.87131253e-02  7.93049559e-02
   3.74480151e-02  2.65276269e-03  3.93749028e-02 -7.09840655e-03
   5.93614392e-02  3.15369815e-02  6.00980483e-02 -5.29052839e-02
   4.06067632e-02 -2.59308442e-02  2.98427492e-02  1.12691964e-03
   7.35149011e-02 -5.03818765e-02 -1.22386552e-01  2.37028468e-02
   2.97264513e-02  4.24768217e-02  2.56337356e-02  1.99518423e-03
  -5.69191538e-02 -2.71598548e-02 -3.29036042e-02  6.60248846e-02
   1.19007066e-01 -4.58791181e-02 -7.26214796e-02 -3.25840600e-02
   5.23413830e-02  4.50552776e-02  8.25312082e-03  3.67024317e-02
  -1.39414473e-02  6.53919280e-02 -2.64272578e-02  2.06382014e-04
  -1.36643536e-02 -3.62810232e-02 -1.95043292e-02 -2.89738793e-02
   3.94270271e-02 -8.84091035e-02  2.62432033e-03  1.36714717e-02
   4.83063571e-02 -3.11565548e-02 -1.17329165e-01 -5.11689968e-02
  -8.85288864e-02 -2.18962468e-02  1.42985815e-02  4.44168746e-02
  -1.34815443e-02  7.43392855e-02  2.66382117e-02 -1.98762231e-02
   1.79190

In [48]:
# import
from langchain.embeddings.sentence_transformer import SentenceTransformerEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import Chroma
from langchain.document_loaders import TextLoader
from langchain.llms import OpenAI

loader = TextLoader("/Users/ashwin.gangadhar/projects/mdb-bfsi-genai/notebooks/temp.txt")
documents = loader.load()
text_splitter = CharacterTextSplitter(chunk_size=2000, chunk_overlap=200)
docs = text_splitter.split_documents(documents)
embedding_function = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")
vectorstore = Chroma.from_documents(docs, embedding_function)
query = "what can a credit card be used for"
docs = vectorstore.similarity_search(query)
prompt = "User Question:{question}\n Context:{context} \n Instruction:Answer to the user question is present in context, if not answer 'I dont know' \n Output Language:{language}  Answer:"
prompt_template = PromptTemplate.from_template(prompt)
llm = OpenAI(temperature=0, model_name="gpt-3.5-turbo")
llm_chain = LLMChain(llm=llm, prompt=prompt_template)
llm_chain.run({"question": query, "context": '\n'.join(e.page_content for e in docs), "language": "kannada"})

/Users/ashwin.gangadhar/anaconda3/lib/python3.9/site-packages/langchain/llms/openai.py:189: UserWarning: You are trying to use a chat model. This way of initializing it is no longer supported. Instead, please use: `from langchain.chat_models import ChatOpenAI`
  warnings.warn(
/Users/ashwin.gangadhar/anaconda3/lib/python3.9/site-packages/langchain/llms/openai.py:769: UserWarning: You are trying to use a chat model. This way of initializing it is no longer supported. Instead, please use: `from langchain.chat_models import ChatOpenAI`
  warnings.warn(


'ಕ್ರೆಡಿಟ್ ಕಾರ್ಡ್ ಬಳಸಿ ವ್ಯಯ ಮಾಡಲು, ಎಲ್ಲಾ ಸಮಯದಲ್ಲಿ ಎಲ್ಲಾ ಸ್ಥಳಗಳಲ್ಲಿ ಸಾಧ್ಯವಾಗುತ್ತದೆ! ಶಾಪಿಂಗ್, ಪ್ರಯಾಣ, ಚಲನಚಿತ್ರ ಟಿಕೆಟ್ಗಳನ್ನು ಖರೀದಿಸಿ ಮತ್ತು ಇನ್ನಷ್ಟು ಮಾಡಿ. ಪ್ರತಿ ಬಾರಿ ನೀವು ಕೋಟಾಕ್ ಕ್ರೆಡಿಟ್ ಕಾರ್ಡ್ ಬಳಸುವಾಗ ಪ್ರತಿಫಲ ಮತ್ತು ಕ್ಯಾಶ್\u200cಬ್ಯಾಕ್ ಗಳನ್ನು ಗಳಿಸಿ.'

[Document(page_content="Why Credit Cards?\nA credit card enables payments, anytime, anywhere! Shop, travel, buy movie tickets & do much more. The list is endless! What's more? Earn rewards and cashback whenever you use Kotak credit cards.\nGet an instant Credit Card in just 3 easy steps!\ncc-step-1\nStep 1: Select Your Ideal Credit Card\nChoose a Credit Card based on your interests, such as Movies, Travel, Shopping, and more\n\ncc-step-2\nStep 2: Share Your Basic Information\nSimply share your basic details like your Name, Phone Number, PAN, and Aadhar\n\ncc-step-3\nStep 3: Complete VKYC and Receive Your Virtual Card\nFinish the VKYC process, receive your virtual card, and start exploring irresistible offers on your new Credit Card", metadata={'source': '/Users/ashwin.gangadhar/projects/mdb-bfsi-genai/notebooks/temp.txt'})]


Created a chunk of size 399, which is longer than the specified 100
Created a chunk of size 124, which is longer than the specified 100


[Document(page_content="Why Credit Cards?\nA credit card enables payments, anytime, anywhere! Shop, travel, buy movie tickets & do much more. The list is endless! What's more? Earn rewards and cashback whenever you use Kotak credit cards.\nGet an instant Credit Card in just 3 easy steps!\ncc-step-1\nStep 1: Select Your Ideal Credit Card\nChoose a Credit Card based on your interests, such as Movies, Travel, Shopping, and more", metadata={'source': '/Users/ashwin.gangadhar/projects/mdb-bfsi-genai/notebooks/temp.txt'}), Document(page_content='cc-step-2\nStep 2: Share Your Basic Information\nSimply share your basic details like your Name, Phone Number, PAN, and Aadhar', metadata={'source': '/Users/ashwin.gangadhar/projects/mdb-bfsi-genai/notebooks/temp.txt'}), Document(page_content='cc-step-3\nStep 3: Complete VKYC and Receive Your Virtual Card\nFinish the VKYC process, receive your virtual card, and start exploring irresistible offers on your new Credit Card', metadata={'source': '/Users/

Number of requested results 4 is greater than number of elements in index 3, updating n_results = 3


'ಕ್ರೆಡಿಟ್ ಕಾರ್ಡ್ ಬಳಸಲು ಹೇಗೆ ಬಳಸಬಹುದು ಎಂದು ನೀವು ಕೇಳಿದ್ದೀರಿ. ಕ್ರೆಡಿಟ್ ಕಾರ್ಡ್ ಬಳಸಿ ನೀವು ಯಾವುದೇ ಸಮಯದಲ್ಲಿ ಯಾವುದೇ ಪಾವತಿಗಳನ್ನು ಮಾಡಬಹುದು! ಶಾಪಿಂಗ್ ಮಾಡಿ, ಪ್ರಯಾಣ ಮಾಡಿ, ಚಲನಚಿತ್ರ ಟಿಕೆಟ್ಗಳನ್ನು ಖರೀದಿಸಿ ಮತ್ತು ಇನ್ನಷ್ಟು ಮಾಡಿ. ಈ ಪಟ್ಟಿ ಅನಿವಾರ್ಯ! ಇದಲ್ಲದೆ, ನೀವು ಕೋಟಾಕ್ ಕ್ರೆಡಿಟ್ ಕಾರ್ಡ್ ಬಳಸುವಾಗ ಪ್ರತಿಫಲ ಮತ್ತು ನಗದು ಪಡೆಯಿರಿ. \n3 ಸುಲಭ ಹೆಜ್ಜೆಗಳಲ್ಲಿ ತಕ್ಷಣ ಕ್ರೆಡಿಟ್ ಕಾರ್ಡ್ ಪಡೆಯಿರಿ!\nಹೆಜ್ಜೆ 1: ನಿಮ್ಮ ಆಸಕ್ತಿಗೆ ಅನುಸಾರವಾಗಿ ಕ್ರೆಡಿಟ್ ಕಾರ್ಡ್ ಆಯ್ಕೆಮಾಡಿ\nಚಲನಚಿತ್ರಗಳು, ಪ್ರಯಾಣ, ಶಾಪಿಂಗ್ ಮುಂತಾದವುಗಳ ಆಧಾರದ ಮೇಲೆ ಕ್ರೆಡಿಟ್ ಕಾರ್ಡ್ ಆಯ್ಕೆಮಾಡಿ\nಹೆಜ್ಜೆ 3: VKYC ಪೂರ್ಣಗೊಳಿಸಿ ಮತ್ತು ನಿಮ್ಮ ವರ್ಚುವಲ್ ಕಾರ್ಡ್ ಪಡೆಯಿರಿ\nVKYC ಪ್ರಕ್ರಿಯೆಯನ್ನು ಮುಗಿಸಿ, ನಿಮ್ಮ ವರ್ಚುವಲ್ ಕಾರ್ಡ್ ಪಡೆಯಿರಿ ಮತ್ತು ನಿಮ್ಮ ಹೊಸ ಕ್ರೆಡಿಟ್ ಕಾರ್ಡ್ ಮೇಲೆ ಆಕರ್ಷಕ ಆಫರ್ಗಳನ್ನು ಅನ್ವೇಷಿಸಲು ಪ್ರಾರಂಭಿಸಿ\nಹೆಜ್ಜೆ 2: ನಿಮ್ಮ ಮೂಲ ಮಾಹಿತಿಯನ್ನು ಹಂಚಿಕೊಳ್ಳಿ\nನಿಮ್ಮ ಹೆಸರು, ಫೋನ್ ಸಂಖ್ಯೆ, ಪ್ಯಾನ್ ಮತ್ತು ಆಧಾರ್ ಮೊದಲಾದ ನಿಮ್ಮ ಮೂಲ ವಿವರಗಳನ್ನು ಹಂಚಿಕೊಳ್ಳಿ.'